In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
#import matplotlib.pyplot as plt
#%matplotlib inline
from geopandas import GeoSeries
import os

In [2]:
def extract_profile(ds,geom,dl,reverse=True):
    """
    Uses an input rasterio dataset, a geometry in the form of Geoseries (polyline) and distance between points
    to return the values of distance along the profile and the value from the dataset
    """
    # reproject the shapefile to raster projection
    geom = geom.to_crs(ds.crs)
    #extracting point coordinates from the line at the specified distance
    distance = 0
    len = geom.iloc[0].length
    while (distance<=(len)):
        pt_geom = geom.interpolate(distance)
        dicti = {'distance':distance,'geometry':pt_geom,'x':pt_geom.x.values,'y':pt_geom.y.values}
        df = gpd.GeoDataFrame(dicti)
        if distance==0:
            df_f = df.copy()
        else:
            df_f = df_f.append(df)
        distance = distance+dl
    if reverse:
        f_d = df_f['distance'].iloc[-1]
        df_f['distance'] = np.abs(df_f['distance']-f_d)
    #sampling the raster dataset values from the obtained point coordinates     
    X = df_f.x.values
    Y = df_f.y.values
    xy = np.vstack((X,Y)).T
    sampled_values = np.array(list(ds.sample(xy)))
    return df_f['distance'].values, np.ma.fix_invalid(np.reshape(sampled_values,np.shape(sampled_values)[0]))
    #return df_f['distance'].values, np.reshape(sampled_values,np.shape(sampled_values)[0])

In [3]:
def sample_values(ds,geom):
    """
    sample values from raster at the given ICESat-2 points
    using nearest neighbour algorithm
    """
    # reproject the shapefile to raster projection
    geom = geom.to_crs(ds.crs)
    #limit points within dataset bound
    x_min,y_min,x_max,y_max = ds.bounds
    #filter geom outside bounds 
    geom = geom.cx[x_min:x_max,y_min:y_max]
    X = geom.geometry.x.values
    Y = geom.geometry.y.values
    xy = np.vstack((X,Y)).T
    sampled_values = np.array(list(ds.sample(xy)))
    return geom.x_atc.values, np.ma.fix_invalid(np.reshape(sampled_values,np.shape(sampled_values)[0]))